In [2]:
import os, sys

cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))

sys.path.insert(0, os.path.join(cwd, "../"))
sys.path.insert(0, os.path.join(cwd, "../student_life/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/bin/"))

import copy
from scipy import stats
import pickle
import importlib
import src.utils.student_utils as student_utils
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances as euclidean_distance

import src.definitions as definitions
import pandas as pd
import numpy as np
from IPython.display import display
from src.data_manager import student_life_var_binned_data_manager
importlib.reload(student_life_var_binned_data_manager)
importlib.reload(student_utils)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

fig_loc = "/Users/nsimsiri/Desktop/histograms"
print(module_path)
cov_path = module_path + "/data/surveys_and_covariates"
print(cov_path)

Student ID couldn't be converted to Integer!
/Users/nsimsiri/Documents/code/ml/MultiRes/student_life
/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data/surveys_and_covariates


/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/notebooks/../src/utils/read_utils.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_file = yaml.load(ymlfile)


In [10]:
data = {}
for i, path in enumerate(os.listdir(cov_path)):
    data[i] = pd.read_csv(cov_path + "/" + path)
    print(i, path)

0 high_lelvel_aggregated_data.csv
1 processed_student_deadlines.csv


In [12]:
agg = data[0]
display(agg[:10])

,student_id,avg_hours_slept,mode_sleep_rating,avg_dead_line_per_week,E_pre,A_pre,C_pre,N_pre,O_pre,E_post,A_post,C_post,N_post,O_post
0,1,7.222222,1.592593,0.909091,62.5,75.555556,88.888889,55.0,78.0,60.0,75.555556,84.444444,57.5,74.0
1,2,7.760000,2.000000,1.090909,42.5,60.000000,84.444444,70.0,64.0,37.5,60.000000,77.777778,65.0,60.0
2,4,6.515152,2.060606,0.636364,47.5,62.222222,53.333333,82.5,58.0,55.0,60.000000,53.333333,92.5,62.0
3,5,7.400000,1.600000,1.545455,62.5,75.555556,88.888889,55.0,66.0,60.0,75.555556,84.444444,47.5,68.0
4,7,7.235294,1.882353,1.818182,62.5,84.444444,75.555556,45.0,46.0,65.0,82.222222,62.222222,57.5,54.0
5,8,7.000000,1.902439,1.636364,52.5,75.555556,84.444444,65.0,66.0,NaN,NaN,NaN,NaN,NaN
6,9,8.000000,1.666667,0.636364,47.5,91.111111,53.333333,42.5,66.0,52.5,95.555556,62.222222,40.0,72.0
7,10,7.840909,2.113636,1.636364,40.0,26.666667,95.555556,67.5,72.0,45.0,26.666667,100.000000,62.5,80.0
8,12,7.450000,1.750000,2.454545,57.5,71.111111,73.333333,57.5,68.0,NaN,NaN,NaN,NaN,NaN
9,14,6.966667,1.666667,2.272727,65.0,73.333333,66.666667,50.0,66.0,65.0,77.777778,73.333333,45.0,68.0


In [13]:
dead = data[1]
display(dead)

,Unnamed: 0,1,2,3,4,5,7,8,9,10,...,47,49,50,51,52,53,54,57,58,59
0,2013-03-27,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1,2013-03-28,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2013-03-29,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,1,0,0,1
3,2013-03-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013-03-31,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
5,2013-04-01,0,0,0,0,0,1,0,0,0,...,1,2,1,0,0,0,0,1,0,2
6,2013-04-02,0,1,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0
7,2013-04-03,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,1,0,0,0
8,2013-04-04,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,2013-04-05,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,1,0,0,2
